## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Some Functions

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Find lane lines on the image function
In the next function we process each image to find lane lines on it. Processing is done with
the folowing steps:

1. Image is gray-scaled
1. Image is blurred. It happened that algorithm works pretty well with blur values between 5 and 13
1. Canny algorithm is applied to the blurred image to get gradient picture. To deal with sunny area on the road on video `chalenge.mp4` canny parameters have to be set low and some additional filtering is applied later.
1. Area of interest is chosen on the image
1. Hough lines procedure was applied to the image. Max line gap parameter was sat so to track dash lane line as solid line. Other parameters was set experimentally.
1. Lines are combined to groups with similar slope angle.
1. For each group average line is calculated, extended to expected edges and removed if it's slope angle is too high
1. If result lines quantity is higher that 2, the distance between each pair of lines is calculated. The best pair is expected to be pair with closest distance value to some fixed parameter.

In [ ]:
def process_image(image):
    
    # Make image grayscale
    image_grayscale = grayscale(image)
    
    # Blur image
    image_blured = gaussian_blur(image_grayscale, 5)
    
    # Get image gradients
    image_grad = canny(image_blured, 40, 60)
    
    # Choose image region where lines are probably located
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(imshape[1]*0.45, imshape[0]*0.62), (imshape[1]*0.55, imshape[0]*0.62), (imshape[1],imshape[0])]], dtype=np.int32)
    image_roi = region_of_interest(image_grad, vertices)
    
    # Apply hough lines procedure
    rho = 1
    theta = np.pi/180
    threshold = 40
    min_line_length = 80
    max_line_gap = 140
    lines = cv2.HoughLinesP(image_roi, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    image_withLines = np.copy(image)
    draw_lines(image_withLines, lines)

    # Filter array with lines to get two target lines
    
    # Combine simular lines
    # make array with lines and line angles
    line_angles = np.array([[l, math.atan2(l[0][0]-l[0][2],l[0][1]-l[0][3])] for l in lines])
    # sort array by angle value
    line_angles_sorted = line_angles[line_angles[:,1].argsort()]
    # groupe lines in groups with close angle values
    line_angle_groups = []
    line_angle_group = []
    prev_angle = np.pi
    for l in line_angles_sorted:
        if abs(prev_angle - l[1]) > 0.1:
            if line_angle_group:
                line_angle_groups.append(line_angle_group)
                line_angle_group = []
        prev_angle = l[1]
        line_angle_group.append(l)
    line_angle_groups.append(line_angle_group)
    
    # calculate avverage line end points for each group, extend result lines to expected edges
    # and remove group if there slope angle is too high
    lane_lines = np.empty([0,4])
    for g in line_angle_groups:
        # get avverage start and end points
        mx1 = np.mean([l[0][0][0] for l in g])
        my1 = np.mean([l[0][0][1] for l in g])
        mx2 = np.mean([l[0][0][2] for l in g])
        my2 = np.mean([l[0][0][3] for l in g])
        # swap them if necessary
        if (my1 < my2):
            mt = my2; my2 = my1; my1 = mt
            mt = mx2; mx2 = mx1; mx1 = mt
        #print(mx1, my1, mx2, my2)
        dh = (mx1-mx2)/ (my1-my2)
        l = math.sqrt((mx1-mx2)**2 + (my1-my2)**2)
        # remove line if it is too short or it's slope angle is too high
        if abs(dh) > 2. or l < 100:
            continue
        # extend line to the desired edges of the image 
        my1 = imshape[0]
        mx1 = dh*(my1-my2)+mx2
        my2 = imshape[0]*0.65
        mx2 = -dh*(my1-my2)+mx1
        #print(dh, mx1, my1, mx2, my2)
        lane_lines = np.append(lane_lines, [[mx1, my1, mx2, my2]], axis=0)
    lane_lines = np.array(lane_lines)
    #print(lane_lines)
    #print(lane_lines[:,4])
    #lane_lines_sorted = lane_lines[lane_lines[:,4].argsort()]
    #lane_lines = np.array([[x[0:4]] for x in lane_lines_sorted[0:2]], dtype='int32')
    
    # Apply additional sorting if more than two lines was tracked.
    # this part calculates distance between all pairs of lines on the low edge of the image and get
    # the pair with value closest to some fixed parameter
    if len(lane_lines) > 2:
        dist_reff = 700
        #lane_lines_bumper_distance has three parameters: abs difference between lines distance and dist_reff;
        #first line index; second line index
        lane_lines_bumper_distance = np.empty([0,3])
        for i in range(len(lane_lines)-1):
            for j in range(i+1, len(lane_lines)):
                dist = abs(lane_lines[i,0]-lane_lines[j,0])
                lane_lines_bumper_distance = np.append(lane_lines_bumper_distance, [[abs(dist-dist_reff), i, j]], axis=0)
        lane_lines_bumper_distance_sorted = lane_lines_bumper_distance[lane_lines_bumper_distance[:,1].argsort()]
        #print(lane_lines_bumper_distance_sorted)
        i1 = int(lane_lines_bumper_distance_sorted[0,1]) #first line index
        i2 = int(lane_lines_bumper_distance_sorted[0,2])
        lane_lines = np.array([[lane_lines[i1]], [lane_lines[i2]]], dtype='int32')
    else:
        lane_lines = np.array([[x[0:4]] for x in lane_lines], dtype='int32')
    
    # debug output
    #if len(lane_lines) != 2:
    #    print("N of lines: ", len(lane_lines))

    # Draw the result!
    image_lines = np.copy(image)*0
    draw_lines(image_lines, lane_lines, thickness=10)
    #draw_lines(image_lines, lines, thickness=10)
    image_final = weighted_img(image_lines, image)
    
    
#    result=cv2.cvtColor(image_grayscale,cv2.COLOR_GRAY2RGB)
#    result=cv2.cvtColor(image_blured,cv2.COLOR_GRAY2RGB)
#    result=cv2.cvtColor(image_grad,cv2.COLOR_GRAY2RGB)
#    result=cv2.cvtColor(image_roi,cv2.COLOR_GRAY2RGB)
#    result=image_withLines
    result=image_final
    
    return result

## Show all files in the test dir

In [ ]:
import os
filesDir = "test_videos/"
allFiles = os.listdir(filesDir)
print(allFiles)
print('Total files: ', len(allFiles))

## Choose file number to process

In [ ]:
n = 2
try:
    fName = os.path.join(filesDir, allFiles[n]) # image file name
    print('Choosed file:', fName)
except IndexError:
    print('Please, choose file number between 0 and ', len(allFiles))

## Run described function on choosen video

In [ ]:
file_output = 'test_videos_output/' + allFiles[n]
clip1 = VideoFileClip(fName)
clip2 = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time clip2.write_videofile(file_output, audio=False)

In [ ]:
iName = 'test_images/report_base.jpg'
image = mpimg.imread(iName)
image_out = process_image(image)
cv2.imwrite('test_images/report_lines.jpg', cv2.cvtColor(image_out, cv2.COLOR_RGB2BGR))